In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
import os


In [2]:

dir_p = r'C:\Users\Honeyz\Desktop\Aessa\THE_SIS\PyroDataProcessed.csv'
raw_dataset = pd.read_csv(dir_p, skipinitialspace=True)

dataset = raw_dataset.copy()
dataset = dataset.dropna()
dataset

,C% Biomass,H% Biomass,O% Biomass,N% Biomass,C% HDPE,H% HDPE,O% HDPE,N% HDPE,C% PS,H% PS,O% PS,N% PS,Time,Rate,Ratio,Temp,Oil%,Char%,Gas%
0,63.85,8.6,3.74,0.33,85.72,14.28,0,0.0,0.0,0.0,0.0,0,0.5,5,66.7,500,60.156065,10.936392,28.907543
1,63.85,8.6,3.74,0.33,85.72,14.28,0,0.0,0.0,0.0,0.0,0,0.5,5,100.0,500,56.000000,0.000000,44.000000
2,63.85,8.6,3.74,0.33,85.72,14.28,0,0.0,0.0,0.0,0.0,0,0.5,5,50.0,500,45.993999,17.313035,36.692966
3,63.85,8.6,3.74,0.33,85.72,14.28,0,0.0,0.0,0.0,0.0,0,0.5,5,33.3,500,44.038082,23.267154,32.694764
4,63.85,8.6,3.74,0.33,85.72,14.28,0,0.0,0.0,0.0,0.0,0,0.5,5,0.0,500,25.762381,34.435228,39.802391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,58.00,6.3,34.00,2.40,0.00,0.00,0,0.0,91.0,8.8,0.3,0,30.0,100,60.0,550,16.633680,15.340660,13.907280
243,58.00,6.3,34.00,2.40,0.00,0.00,0,0.0,91.0,8.8,0.3,0,30.0,100,61.5,550,15.999231,14.779844,13.749581
244,58.00,6.3,34.00,2.40,0.00,0.00,0,0.0,91.0,8.8,0.3,0,30.0,100,63.0,550,15.349117,14.221099,13.605475
245,58.00,6.3,34.00,2.40,0.00,0.00,0,0.0,91.0,8.8,0.3,0,30.0,100,64.5,550,14.683338,13.664426,13.474963


In [3]:
#The following script divides data into attributes and labels:
X = dataset.iloc[:, :16].values
y = dataset.iloc[:, 16:].values


loss_per_fold_0 = []
loss_per_fold_1 = []
loss_per_fold_2 = []


# Create the SVR regressor
svr = SVR(kernel='linear',epsilon=3)
# Create the Multioutput Regressor
mor = MultiOutputRegressor(svr)

    
#Divide the data into training and testing sets for KFold Cross validaton loop
#Define the K-fold Cross Validator
cv = KFold(n_splits=247, shuffle=False)
cv.get_n_splits(X)
print(cv,"\n")

# K-fold Cross Validation model evaluation
fold_no = 1


for train_index, test_index in cv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Feature Scaling
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    # Train the regressor
    mor = mor.fit(X_train, y_train)

    # Generate predictions for testing data
    y_pred = mor.predict(X_test)


     # Generate generalization metrics
    
    print(f'loss for fold {fold_no}: \n Regressor 1: {metrics.mean_absolute_error(y_test[:,0], y_pred[:,0])}\n Regressor 2:{metrics.mean_absolute_error(y_test[:,1], y_pred[:,1])}\nRegressor 3:{metrics.mean_absolute_error(y_test[:,2], y_pred[:,2])}\n\n')

    loss_per_fold_0.append(metrics.mean_absolute_error(y_test[:,0], y_pred[:,0]))
    loss_per_fold_1.append(metrics.mean_absolute_error(y_test[:,1], y_pred[:,1]))
    loss_per_fold_2.append(metrics.mean_absolute_error(y_test[:,2], y_pred[:,2]))

    # Increase fold number
    fold_no = fold_no + 1

KFold(n_splits=247, random_state=None, shuffle=False) 

loss for fold 1: 
 Regressor 1: 7.106230988546166
 Regressor 2:1.8747229935918046
Regressor 3:3.352033137069082


loss for fold 2: 
 Regressor 1: 8.97034820129366
 Regressor 2:4.1950424593377065
Regressor 3:17.797441567796703


loss for fold 3: 
 Regressor 1: 1.2096032125334304
 Regressor 2:0.057217500171219626
Regressor 3:1.4964599555102396


loss for fold 4: 
 Regressor 1: 2.685097824425661
 Regressor 2:1.568224240569144
Regressor 3:5.467352929481258


loss for fold 5: 
 Regressor 1: 3.9583943135348143
 Regressor 2:4.149331511851489
Regressor 3:4.2917530226827765


loss for fold 6: 
 Regressor 1: 3.7661330416468566
 Regressor 2:4.045000709827278
Regressor 3:4.139886827352704


loss for fold 7: 
 Regressor 1: 2.96738458207572
 Regressor 2:3.8439734784505184
Regressor 3:3.7811483993439126


loss for fold 8: 
 Regressor 1: 2.2459236489868353
 Regressor 2:3.6002876285622882
Regressor 3:3.487234251291042


loss for fold 9: 
 Regresso

In [4]:
print('Average scores for all folds:')
print(f'> Mean loss fisrt regressor: {np.mean(loss_per_fold_0)}')
print(f'> Mean loss fisrt regressor: {np.mean(loss_per_fold_1)}')
print(f'> Mean loss fisrt regressor: {np.mean(loss_per_fold_2)}')
print('------------------------------------------------------------------------')


Average scores for all folds:
> Mean loss fisrt regressor: 5.2948514163919755
> Mean loss fisrt regressor: 2.4271987561501827
> Mean loss fisrt regressor: 2.4422704430032094
------------------------------------------------------------------------
